In [12]:
%load_ext autoreload
%autoreload 2

import os, datetime
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from IPython import display
import IPython
from IPython.display import display

mpl.rcParams['figure.figsize'] = (16, 5)
mpl.rcParams['axes.grid'] = False
pd.options.display.max_rows = 5
%matplotlib inline  

!mkdir gen

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
mkdir: gen: File exists


In [160]:
%%writefile  gen/getstocksdata.py
#!/usr/local/bin/python 

'''
The following code: it is just here to protect my API-KEY code not really useful
for the problem at hand. If your key is in your or update your API_KEY then you may safely ignore 
this code.
'''

def getkey(key='password'):
    API_KEY, lines, file =None, None, os.path.expanduser('~/.keys/keys.json')
    if os.path.exists(file):
        j = json.load(open(file, 'r'))
        return j['AV_API_KEY'], j['NEWSAPI_KEY']

    #alpha_vantage key
    avk = decrypt('baTEje52rx+kAsuAN9PdxMeC03p/HuRVTzskLiso1/c=', key)
    newsk = decrypt('505Db5sDvHvptBPzE8IhsewneuanOKV3gKpN+26lS3A=', key)
    return avk, newsk;
        
def encrypt(msg_text = b'message', secret_key='password'):
    if (type(msg_text) == str):
        msg_text = bytes(msg_text, encoding='utf-8').rjust(32)
    if (type(secret_key) == str):
        secret_key = bytes(secret_key, encoding='utf-8') .rjust(32)

    cipher = AES.new(secret_key,AES.MODE_ECB) 
    encoded = base64.b64encode(cipher.encrypt(msg_text))
    ret = encoded.decode("utf-8")
    print(ret)
    return ret

def decrypt(encoded, secret_key='password'):
    if (type(secret_key) == str):
        secret_key = bytes(secret_key, encoding='utf-8') .rjust(32)

    cipher = AES.new(secret_key,AES.MODE_ECB) 
    if (type(encoded) == str):
        encoded = bytes(encoded, encoding='utf-8')
    decoded = cipher.decrypt(base64.b64decode(encoded))
    ret =decoded.decode("utf-8").strip()
    print(ret)
    return ret

'''
This will read required symbols and saves them to data directory
'''
def save_data(symbol, API_KEY="", check=True):
    from alpha_vantage.timeseries import TimeSeries
    
    outf = f'./data/daily_{symbol}.csv'
    if (check and os.path.exists(outf)):
        print(f"{outf:22} exists, ... nothing to do")
        return;
        
    ts = TimeSeries(key=API_KEY, output_format='pandas')
    data, meta_data = ts.get_daily(symbol, outputsize='full')
    data.insert(0, 'timestamp', value=data.index)

    data.columns = 'timestamp,open,high,low,close,volume'.split(',')
    data.to_csv(outf, index=False)
    return data

'''
Read all the files in data with daily_*, reads them and return a list
'''
def read_data():
    a={}
    for f in glob.glob('data/daily_*'):
        symbol = os.path.basename(os.path.splitext(f)[0]).split("_")[1]
        print(f'Reading {f} Symbol: {symbol}')
        df = pd.read_csv(f)
        df.sort_values(by='timestamp', ascending=False, inplace=True)
        df.index=(range(0,len(df)))
        ncs = ['timestamp'] + [f'{symbol}_{c}' for c in df.columns[1:]]
        df.columns = ncs
        a[symbol] = df
        
    maxrows = min([len(d) for d in a.values()])
    return a, maxrows, df

'''
Combines all the dataframes into one

The problem for us to join multiple index funds from ASIA is that they have different holidays
Therefore we have gaps in the trading days. Therefore 
'''
def combine_data(a, outp="data/stockdata.csv"):
    # Different excahnges have various holidays, therefore values may be missing  
    # Get All Data Frames and their corresponding time stamps
    #
    ar=np.array([d['timestamp'].values for d in a.values()])
    at=np.concatenate(ar)
    at=set(at)
    af = pd.DataFrame()
    af['timestamp'] = list(at);

    for k,v in a.items():
        print(f"Getting {k:32} \r", end='')
        af=pd.merge(af,v, how="left", left_on="timestamp".split(), right_on="timestamp".split())
    print()
    af.sort_values(by='timestamp', ascending=False, inplace=True)
    af = af.reset_index(drop=True)
    af = af.fillna(method='ffill' ).fillna(method='bfill')
    af.to_csv(outp, index=False)
    return af

def needNewData(stockfile="data/stockdata.csv"):
    file= stockfile
    c1 = os.path.exists(file)
    c2 = os.stat(file)
    t1 = pd.to_datetime(c2.st_mtime, unit="s", utc=False)
    t1.day
    t2 = pd.to_datetime(datetime.datetime.now())
    wd = t2.weekday() < 5  # means not saturday or sunday

    tdf = t2 - t1
    if ( (wd and tdf.seconds < 4) ):
        print(f"{file} is up to date {t1.day}, {t2.day} ")
        return False;
    print(f'Fetch new set of data')
    return True

def getdata(symbs='MSFT GLD GOOGL SPX AAPL IBM' , dontforce=False):
    
    stockfile="../data/stockdata.csv"
    if (not os.path.exists(stockfile)):
        stockfile="data/stockdata.csv"
        
    stockfile="data/stockdata.csv"
    if (not dontforce and not os.path.exists(stockfile)):
        API_KEY = None  # Put your API KEY if you need to test data download or just use the data
        API_KEY, NEWS_API_KEY = API_KEY,"" or getkey()

        for f in symbs.split():
            save_data(f, API_KEY=API_KEY)

        ASIA='''TOKYO 6758.T HITACHI 6501.T HNGKNG 0168.HK SHANGAI 601288.SS SHNZEN'''
        s=ASIA.split()
        ASIA = {k[0]:k[1] for k in   zip(s[0::2], s[1::2])}

        for k, v in ASIA.items():
            print(f'getting data for {k} => symbol {v}')
            save_data(v, API_KEY)

        a, maxrows, ldf  = read_data()
        nf= combine_data(a, stockfile)
    
    
#-----------------------------------------------------------------------------------
#*** NOTE: DO NOT EDIT THIS FILE - THIS iS CREATED FROM: inv_utils.ipynb
def inJupyter():
    try:    get_ipython; return True
    except: return False
#-----------------------------------------------------------------------------------
if __name__ == '__main__':
    if (not inJupyter()):
        t1 = datetime.datetime.now()
        getdata()
        t2 = datetime.datetime.now()
        print(f"#All Done in {str(t2-t1)} ***")
    else:
        pass

Overwriting gen/getstocksdata.py


In [452]:
%%writefile  gen/dataconfig.py
#!/usr/local/bin/python 

import re, sys, os, datetime, getopt, glob, argparse, datetime, json, base64, pickle
import numpy as np
import pandas as pd
from collections import defaultdict
from sklearn.preprocessing import StandardScaler

#~~~~ Find any sensor highly correlated with time and drop them.
def detectTimeCorrelated(df, val=0.94):
    timeCorSensors = []
    if ( not 'time' in df.columns): #assume first column is time column
        dcols = ['time'] + [c for c in df.columns[1:]]
        df.columns = dcols
    
    timeser = pd.Series(df[['time']].values.reshape(-1))
    if ( timeser.dtype != np.number ):
        timeser = pd.to_datetime(timeser).astype(int)
    
    
    DROP_INDEX = 0;
    for sensor in df.columns:
        if (sensor == 'time'):
            continue;
        #print(f"#Testing {sensor}...")
        sensorSeries = pd.Series(df[sensor].values.reshape(-1))
        for i in range(8):
            c1 = timeser[i:].corr(sensorSeries[i:])
            c2 = timeser[i:].corr(sensorSeries[:-i])
            if np.abs(c1) >= val or np.abs(c2) >= val:
                timeCorSensors.append(sensor)
                DROP_INDEX = max(DROP_INDEX, i) #lets drop first few rows
                break;
                
    if ( len(timeCorSensors) > 0):
        print(f"#Time Cor: #{len(timeCorSensors)}, #Shape before:{df.shape}")
        #df.drop(timeCorSensors, axis=1, inplace=True)
        #df = df[DROP_INDEX:]
        #print(f"#After dropping: {DROP_INDEX} =>{df.shape}")
        
    return timeCorSensors
#-----------------------------------------------------------------------------------
def precheck(df):
    cols = df.columns[df.dtypes.eq('object')]
    if (len(cols) > 0):
        print(f"WARNING: *** Non numeric columns => {cols}")
        return 0
    return 1
#-----------------------------------------------------------------------------------
def detectCols(file, nUnique=4, tcoeff=0.92):
    tf1 = file
    if (type(tf1) == str):
        tf1 = pd.read_csv(tf1, comment="#")
    
    #Lets check if it has any non-numeric columns! Warning
    precheck(tf1)
    
    unique_vals  = tf1.nunique()
    constantCols = unique_vals[ unique_vals == 1].index                             # constant Columns
    onehotECols  = unique_vals[(unique_vals > 2 ) & (unique_vals<=nUnique)].index   # Categorical Columns
    categorCols  = unique_vals[(unique_vals >=2 ) & (unique_vals <= nUnique)].index # Categorical Columns
    binaryCols   = unique_vals[(unique_vals == 2)].index                      # Binary

    numericCols  = tf1.select_dtypes(include=np.number).columns           # numerics
    numericCols  = [c for c in numericCols if c not in categorCols]
    numericCols  = [c for c in numericCols if c not in constantCols]
    notNumerics  = tf1.select_dtypes(exclude=np.number).columns           # non - numerics
    notNumerics  = [c for c in notNumerics if c not in categorCols]       # non - numerics

    try:
        timeCorCols  = detectTimeCorrelated(tf1, tcoeff)
    except:
        timeCorCols = []
        
    ret = {
            "file"          : [file] if (type(file) == str) else ["??"],
            "number_Unique" : nUnique            , 
            "constantCols"  : list(constantCols ),   # No Signals
            "categorCols"   : list(categorCols  ),   # Categorical Columns
            "onehotECols"   : list(onehotECols  ),   # Cats > 2 and < Unique Values
            "binaryCols"    : list(binaryCols   ),   # Binary
            "numericCols"   : list(numericCols  ),  
            "notNumerics"   : list(notNumerics  ),
            "timeCorrelated": tcoeff             ,   # Time correlated
            "timeCorrCols"  : list(timeCorCols  ),   # Time correlated Columns
            "timeCorrLength": len(timeCorCols  ) ,   # Time correlated Columns length
            "excludePattern": [] , #Exclude patterns
            "includePattern": [] , #include patterns
            "dropColumns"   : [],
            "finalColumns"  : ["binaryCols", "numericCols", "onehotECols"],
            "scalerString"  : [],
            "addDiffs"      : [],
            "train_pct"     : 100,
            "train_count"   : len(tf1),
        }
    return ret, tf1;
#-----------------------------------------------------------------------------------
'''
This will scale the numeric columns - if this changes - you need to use dataprep
'''
def scaleNumerics(tf1, numericCols=[], train_pct=0.8, config={}):
    ltf = tf1[numericCols];
    trn_split_count = int(len(ltf) * train_pct) if (train_pct < 1) else train_pct
    
    scaler = StandardScaler()
    dfs = scaler.fit(ltf[:trn_split_count])
    ltf = (ltf-dfs.mean_)/dfs.scale_

    scalerstr = base64.b64encode(pickle.dumps(scaler, protocol=None, fix_imports=True)).decode("utf-8")
    config["scalerString"] = [scalerstr]
    config["train_pct"   ] = train_pct
    config["train_count" ] = trn_split_count
    
    return ltf, scaler, trn_split_count
#-----------------------------------------------------------------------------------
#*** NOTE: DO NOT EDIT THIS FILE - THIS iS CREATED FROM: inv_utils.ipynb
def inJupyter():
    try:    get_ipython; return True
    except: return False

def formatConfig(out: dict):
    #outj = json.dumps(out, sort_keys=False, indent=None)
    outj = f"[START]\n{{\n"
    for k,v in out.items():
        kk = f'"{k}"'
        outj += f'{kk:>20}: {v},\n'
    outj += '"end": 0 \n}\n[END]\n'
    
    return outj
    
def process():
    n  = len(sysargs.input_files)
    un = sysargs.unique
    tc = sysargs.tcoeff
    for i, file1 in enumerate(sysargs.input_files):
        print(f"#=>Processing {i+1}/{n} {file1} #unique: {un} tcoeff: {tc} - standby")
        out,df = detectCols(file1, un, tc)
        
        trnSplit = sysargs.tsplit
        scaleNumerics(df, out["numericCols"], trnSplit, out)
        
        #Prepare to save to file
        out['file'] = [os.path.abspath(file1)];
        outj = formatConfig(out)
        break;
    print(outj)
    return outj
    
#-----------------------------------------------------------------------------------
sysargs=None
def addargs():
    sysargs = None
    p = argparse.ArgumentParser(f"{os.path.basename(sys.argv[0])}:")
    p.add_argument('-u', '--unique', type=int,   default=6,    help="# of unique values!")
    p.add_argument('-t', '--tcoeff', type=float, default=0.94, help="# Time Correlation value Sensors!")
    p.add_argument('-s', '--scale',  type=bool,  default=1,    help="# Find Scaler!")
    p.add_argument('-p', '--tsplit', type=float, default=0.7,  help="# Find training Split!")
    p.add_argument('args', nargs=argparse.REMAINDER)
    p.add_argument('input_files',action="store", type=str, nargs='+', help="input file(s)")

    #p.print_help() # always print -help
    try:
        sysargs=p.parse_args(sys.argv[1:])
        #print(f'using:\n{sysargs}')
    except argparse.ArgumentError as exc:
        #par.print_help()
        print(exc.message )
        
    return sysargs
#-----------------------------------------------------------------------------------
if __name__ == '__main__':
    if (not inJupyter()):
        t1 = datetime.datetime.now()
        sysargs = addargs()
        process()
        t2 = datetime.datetime.now()
        print(f"#All Done in {str(t2-t1)} ***")
    else:
        pass

Overwriting gen/dataconfig.py


In [499]:
#%%writefile  gen/dataprepare.py
#!/usr/local/bin/python 

import re, sys, os, datetime, getopt, glob, argparse, datetime, json, base64, pickle
import numpy as np
import pandas as pd
from collections import defaultdict
from sklearn.preprocessing import StandardScaler
sys.path.append(".")
sys.path.append("gen")
import dataconfig;

'Covert to one_hot encoding with prefix for columns'
def makeOneHot(tf1, oheCols=[]):
    ohe = pd.DataFrame();
    for c in oheCols:
        one_hot = pd.get_dummies(tf1[c])
        nc = [f'{c}__{k}' for k in one_hot.columns]
        one_hot.columns = nc
        ohe = pd.concat([ohe, one_hot], axis=1)

    return ohe

def addDiff(tf1, col):
    #col = "MSFT_open"
    tf1[f'{col}_diff1'] = tf1[col] - tf1[col].shift(-1)
    return tf1

#-----------------------------------------------------------------------------------
'''
This will scale the numeric columns - if this changes - you need to use dataprep
'''
def scaleNumerics(tf1, numericCols=[], train_pct=0.8, config={}):
    ltf = tf1[numericCols];
    trn_split_count = int(len(ltf) * train_pct) if (train_pct < 1) else train_pct
    
    scaler = StandardScaler()
    dfs = scaler.fit(ltf[:trn_split_count])
    ltf = (ltf-dfs.mean_)/dfs.scale_

    scalerstr = base64.b64encode(pickle.dumps(scaler, protocol=None, fix_imports=True)).decode("utf-8")
    config["scalerString"] = [scalerstr]
    config["train_pct"   ] = train_pct
    config["train_count" ] = trn_split_count
    
    return ltf, scaler, trn_split_count
#-----------------------------------------------------------------------------------
def formatConfig(out: dict):
    #outj = json.dumps(out, sort_keys=False, indent=None)
    outj = f"[START]\n{{\n"
    for k,v in out.items():
        kk = f'"{k}"'
        outj += f'{kk:>20}: {v},\n'
    outj += '"end": 0 \n}\n[END]\n'
    
    return outj
#-----------------------------------------------------------------------------------
def getJSONconfig(cf = "config*"):
    with open(cf, "r") as f:
        cf = f.read()

    r1=re.findall("\[START](.*)\[END]", cf, flags=re.MULTILINE|re.DOTALL)
    if ( len(r1) <= 0):
        print(f"Ignoring: Configuration not found in {cf}! no worries")
        return None
    r1 = r1[0].replace("'", '"')    
    rj = json.loads(r1)
    return rj
    
def getconfig(cf = "config*"):
    confFiles = sorted(glob.glob(cf))
    if ( len(confFiles) <= 0):
        print(f"No Configuration files {cf} found!!!")
        exit(1)

    # Read and merge the configuration files
    ret = {}
    for cf in confFiles:
        print(f"#Getting Configuration from {cf}")
        rj = getJSONconfig(cf)
        if (rj):
            ret.update(rj)
        
    scalerStr= ret.get('scalerString', [])
    if len(scalerStr) > 0:
        scalerStr = scalerStr[0]
        decoded = base64.b64decode(scalerStr)
        scaler = pickle.loads(decoded,fix_imports=True)
        ret['scaler'] = scaler
    else:
        ret['scaler'] = []
        
    return ret
#-----------------------------------------------------------------------------------
def process(config, input_files, outputFile=None):
    conf = getconfig(config)
    
    n  = len(input_files)
    adf = pd.DataFrame();
    for i, file1 in enumerate(input_files):
        print(f"=>Processing {i+1}/{n} {file1} - standby")
        df = pd.read_csv(file1, comment='#')
        
        # STEP 0: Add Differecing Columns
        cdiffs = conf.get('addDiffs', [])
        for c in cdiffs:
            addDiff(df, c)

        drps = conf.get("dropColumns", [])
        df.drop(drps, inplace=True, errors="ignore")
        df.sort_values(by=df.columns[0], ascending=True, inplace=True)
        df = df.reset_index(drop=True)
            
        # STEP 1. --> Scale all Numeric Columns if scaler exists
        dfn= df[conf['numericCols']]    # Get all Numeric Columns
        
        scale  = conf.get('scale', 0)
        scaler = conf['scaler']
        
        if ( scale and scaler ):                  # I know we are assuming Standard Scaler now
            dfn = (dfn-scaler.mean_)/scaler.scale_
        if ( scale and not scaler ):              # Find a new Scaler
            dfn, scaler, trn_split_count = scaleNumerics(dfn, dfn.columns, conf['train_pct'], conf)
            
        # STEP 2: => One hot encode 
        ohe = None
        if len(conf['onehotECols']) > 0:
            ohe=makeOneHot(df, conf['onehotECols'])
        
        # STEP 3: Get Binary columns
        binaryCols= conf['binaryCols']
        
        # => Todo: I Know I Should account for Exclude Patterns here
        
        #FINALLY: Concatenate all the data
        ndf= pd.concat([df[binaryCols], dfn, ohe, ], axis=1)
        adf= pd.concat([adf, ndf], axis=0)
        
        break; # Will do only one file now - fix it later
    
        
    # Lets Do one final clean up 
    adf.dropna(inplace=True)
    adf.reset_index(inplace=True, drop=True)

    conf['resultCols'] = [c for c in adf.columns]
    outj = formatConfig(conf)
    print(outj)
    
    if(outputFile):
        adf.to_csv(outputFile, index=False)
        
    return conf, df, adf;

#-----------------------------------------------------------------------------------
sysargs=None
def addargs():
    sysargs = None
    p = argparse.ArgumentParser(f"{os.path.basename(sys.argv[0])}:")
    p.add_argument('-c', '--config', type=str, default="config.txt", help="Config Files")
    p.add_argument('-o', '--output', type=str, default="out.csv", help="output file")
    p.add_argument('args', nargs=argparse.REMAINDER)
    p.add_argument('input_files',action="store", type=str, nargs='+', help="input file(s)")

    #p.print_help() # always print -help
    try:
        sysargs=p.parse_args(sys.argv[1:])
    except argparse.ArgumentError as exc:
        print(exc.message )
        
    return sysargs
#-----------------------------------------------------------------------------------
#*** NOTE: DO NOT EDIT THIS FILE - THIS iS CREATED FROM: inv_utils.ipynb
def inJupyter():
    try:    get_ipython; return True
    except: return False

#-----------------------------------------------------------------------------------
if __name__ == '__main__':
    if (not inJupyter()):
        t1 = datetime.datetime.now()
        sysargs = addargs()
        ret = process(sysargs.config, sysargs.input_files, sysargs.output)
        t2 = datetime.datetime.now()
        print(f"#All Done in {str(t2-t1)} ***")
    else:
        pass

In [495]:
f='/opt/SCHAS/NNBook/notebooks/NNetworks/LSTM/data/stockdata_ext.csv'
#df=pd.read_csv(f)
#df=df['MSFT_open', 'MSFT_high', 'MSFT_low', 'MSFT_close', 'MSFT_volume']
#df.sort_values(by=df.columns[0], inplace=True)
#df

In [501]:
config="gen/myconfig"
conf, df, adf = process(config, ['/opt/SCHAS/NNBook/notebooks/NNetworks/LSTM/data/stockdata_ext.csv'], "out.csv")
df[0:4]

#Getting Configuration from gen/myconfig
=>Processing 1/1 /opt/SCHAS/NNBook/notebooks/NNetworks/LSTM/data/stockdata_ext.csv - standby
[START]
{
              "file": ['/opt/SCHAS/NNBook/notebooks/NNetworks/LSTM/data/stockdata_ext.csv'],
     "number_Unique": 6,
      "constantCols": [],
       "categorCols": ['MSFT_-ve', 'MSFT_+ve'],
       "onehotECols": ['MSFT_+ve'],
        "binaryCols": ['MSFT_-ve'],
       "numericCols": ['MSFT_open', 'MSFT_high', 'MSFT_low', 'MSFT_close', 'MSFT_volume', 'MSFT_open_diff1'],
       "notNumerics": ['timestamp'],
    "timeCorrelated": 0.94,
      "timeCorrCols": [],
    "timeCorrLength": 0,
    "excludePattern": [],
    "includePattern": [],
       "dropColumns": [],
      "finalColumns": ['binaryCols', 'numericCols', 'onehotECols'],
      "scalerString": ['gANjc2tsZWFybi5wcmVwcm9jZXNzaW5nLmRhdGEKU3RhbmRhcmRTY2FsZXIKcQApgXEBfXECKFgJAAAAd2l0aF9tZWFucQOIWAgAAAB3aXRoX3N0ZHEEiFgEAAAAY29weXEFiFgPAAAAbl9zYW1wbGVzX3NlZW5fcQZjbnVtcHkuY29yZS5tdWx0aWFycmF5CnNj

,timestamp,MSFT_-ve,MSFT_+ve,MSFT_open,MSFT_high,MSFT_low,MSFT_close,MSFT_volume,0168.HK_open,0168.HK_high,...,6501.T_high,6501.T_low,6501.T_close,6501.T_volume,6758.T_open,6758.T_high,6758.T_low,6758.T_close,6758.T_volume,MSFT_open_diff1
0,2000-01-03,1,S__1,95.12,97.12,92.81,93.81,33148100.0,2.35,2.500,...,7440.0,7280.0,7380.0,1584800.0,15125.0,15300.0,14680.0,15000.0,4384800.0,NaN
1,2000-01-04,1,S__1,95.12,97.12,92.81,93.81,33148100.0,2.35,2.500,...,7440.0,7280.0,7380.0,1584800.0,15125.0,15300.0,14680.0,15000.0,4384800.0,0.0
2,2000-01-05,1,S__1,95.12,97.12,92.81,93.81,33148100.0,2.45,2.450,...,7440.0,7280.0,7380.0,1584800.0,15125.0,15300.0,14680.0,15000.0,4384800.0,0.0
3,2000-01-06,1,S__1,95.12,97.12,92.81,93.81,33148100.0,2.35,2.425,...,7440.0,7280.0,7380.0,1584800.0,15125.0,15300.0,14680.0,15000.0,4384800.0,0.0


In [478]:
%%writefile  gen/NN1.py
#!/usr/local/bin/python 

import re, sys, os, datetime, getopt, glob, argparse, datetime, json, base64, pickle, glob
import numpy as np
import pandas as pd
from collections import defaultdict
from sklearn.preprocessing import StandardScaler
sys.path.append(".")
sys.path.append("gen")
import dataconfig;
import keras
import tensorflow as tf
import numpy as np
from sklearn import preprocessing
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (16, 5)
mpl.rcParams['axes.grid'] = False
pd.options.display.max_rows = 5
%matplotlib inline  

use_keras=1
if ( use_keras):
    from keras.models import Sequential, Model
    from keras.layers import Dense, Dropout, LSTM, Input, Activation, concatenate, TimeDistributed
    from keras.layers import Conv1D, GlobalMaxPool1D,Flatten, Bidirectional, RepeatVector, MaxPooling1D
    from keras.preprocessing.sequence import TimeseriesGenerator
    from keras import regularizers
    from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, Callback
    from keras import optimizers
    from keras.models import load_model
else:
    from tensorflow.keras.models import Sequential, Model
    from tensorflow.keras.layers import Dense, Dropout, LSTM, Input, Activation, concatenate, TimeDistributed
    from tensorflow.keras.layers import Conv1D, GlobalMaxPool1D,Flatten, Bidirectional, RepeatVector, MaxPooling1D
    from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
    from tensorflow.keras import regularizers
    from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, Callback
    from tensorflow.keras import optimizers
    from tensorflow.keras.models import load_model

#-----------------------------------------------------------------------------------
def getJSONconfig(cf = "config*"):
    with open(cf, "r") as f:
        cf = f.read()

    r1=re.findall("\[START](.*)\[END]", cf, flags=re.MULTILINE|re.DOTALL)
    if ( len(r1) <= 0):
        print(f"Ignoring: Configuration not found in {cf}! no worries")
        return None
    r1 = r1[0].replace("'", '"')    
    rj = json.loads(r1)
    return rj
#-----------------------------------------------------------------------------------
def process(config, input_files, outputFile=None):
    conf = getconfig(config)
    
    model1=None      

    #sensN = len(self.train_transformed[0].columns)  # number of sensors (eliminating the two time ones)
    #outN = len(self.num_id_list) # number of output sensors; the non-categorical ones        

    lookBack   = tsParam['length']
    nFeatures  = X.shape[1]  # Number of features 
    lstm_OPDim = y.shape[1]  # This is usually all sensors except categorical that to train LSTM on
    lstm_IPDim = 256
    drop       = 0.3
    optimizer  = optimizers.Adam(lr=0.0005)
    loss       = 'mse'
    k_rrizer   = None
    r_rrizer   = None

    input_layer  = Input(shape=(lookBack, nFeatures), dtype='float32', name='input')
    memory_layer = LSTM( lstm_IPDim, return_sequences=True, name="memory1")(input_layer)
    memory_layer = LSTM (int(lstm_IPDim/2), return_sequences=False, name="memory2")(memory_layer)
    repeated     = RepeatVector(lookBack)(memory_layer)
    memory_layer = LSTM (int(lstm_IPDim/2), return_sequences=True, name="first1out")(repeated)
    memory_layer = LSTM (lstm_IPDim,  return_sequences=True, name="first2out")(memory_layer)
    decoded_inputs = TimeDistributed(Dense(units=lstm_OPDim, activation='linear'))( memory_layer)

    #  Try spatial dropout?
    dropout_input = Dropout(drop)(input_layer)
    concat_layer  = concatenate([dropout_input, decoded_inputs])

    #memory_layer = LSTM (units=self.lstm_dim, return_sequences=False)(concat_layer)
    memory_layer = LSTM (units=lstm_IPDim, 
                             kernel_regularizer = k_rrizer, 
                             recurrent_regularizer = r_rrizer, 
                             return_sequences=False)(concat_layer)
    preds = Dense(units=lstm_OPDim, activation='linear')(memory_layer)

    model1 = Model(input_layer, preds)
    model1.compile(optimizer = optimizer, loss= loss)             

    print(model1.summary())
    return conf, df, adf;

#-----------------------------------------------------------------------------------
sysargs=None
def addargs():
    sysargs = None
    p = argparse.ArgumentParser(f"{os.path.basename(sys.argv[0])}:")
    p.add_argument('-c', '--config', type=str, default="config.txt", help="Config Files")
    p.add_argument('-o', '--output', type=str, default="out.csv", help="output file")
    p.add_argument('args', nargs=argparse.REMAINDER)
    p.add_argument('input_files',action="store", type=str, nargs='+', help="input file(s)")

    #p.print_help() # always print -help
    try:
        sysargs=p.parse_args(sys.argv[1:])
    except argparse.ArgumentError as exc:
        print(exc.message )
        
    return sysargs
#-----------------------------------------------------------------------------------
#*** NOTE: DO NOT EDIT THIS FILE - THIS iS CREATED FROM: inv_utils.ipynb
def inJupyter():
    try:    get_ipython; return True
    except: return False

#-----------------------------------------------------------------------------------
if __name__ == '__main__':
    if (not inJupyter()):
        t1 = datetime.datetime.now()
        sysargs = addargs()
        ret = process(sysargs.config, sysargs.input_files, sysargs.output)
        t2 = datetime.datetime.now()
        print(f"#All Done in {str(t2-t1)} ***")
    else:
        pass

In [484]:
js.update({})